# Supabase Test

- prerequisite
    - supabase python package
      ```
      pip install supabase
      ```

- input
    - nlp result info: `testing-nlp-preprocessing_result_info.json`

- result

    - update supabase table `articles`
        - programming_languages
        - keywords_unigram
        - keywords_bigram
        
    - function `prog_lang_freq` for counting Programming Language Frequency

In [1]:
import os
import json

from supabase import create_client, Client

# Load Input
- nlp result info: `testing-nlp-preprocessing_result_info.json`

In [2]:
# load nlp_result_info
nlp_root = os.path.dirname(os.getcwd())

nlp_result_info_path = os.path.join(nlp_root, "testing/testing-nlp-preprocessing_result_info.json")
with open(nlp_result_info_path, 'r') as infile:
    nlp_result_info = json.load(infile)

# print debug info
test_key = list(nlp_result_info.keys())[4]
test_nlp_info = nlp_result_info[test_key]

print(
    json.dumps(
        test_nlp_info, 
        indent=4, 
        ensure_ascii=False
    )
)

{
    "href": "https://ithelp.ithome.com.tw/articles/10282236",
    "title": "[專案上線第01天] -  新來的主管說要寫 Vue Test Utils 單元測試",
    "content_html": "<div class=\"markdown__style\">\n                                                            <h3>前言</h3>\n<blockquote>\n<p>該系列是為了讓看過Vue官方文件或學過Vue但是卻不知道怎麼下手去重構現在有的網站而去規畫的系列文章，在這邊整理了許多我自己使用Vue重構很多網站的經驗分享給讀者們。</p>\n</blockquote>\n<p>什麼？單元測試？當你開始接觸開發專案有一段時間後，你會開始漸漸聽到這個專業術語，就讓我來大家了解一下什麼是單元測試</p>\n<p><iframe width=\"560\" height=\"315\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen=\"allowfullscreen\" src=\"https://www.youtube.com/embed/j2ggBXF54dA\"></iframe><br>\n影片搭配文章看學習效果會更好喔</p>\n<h2>什麼是單元測試？</h2>\n<p>簡單來說程式碼的最小單位進行測試，確保程式邏輯不會在團隊維護的過程中出錯，維護程式碼的品質。所謂的最小單位，我用個例子來舉例，假如你今天有一個主功能是由 A跟Ｂ兩個功能所組成的，而這兩個功能就是我們所說的最小單位，所以在撰寫測試的時候我們重點在針對A跟Ｂ來進行測試，主功能的測試中不會包含 A跟Ｂ的測試，這樣的測試就是我們所說的單元測試。</p>\n<h2>為什麼需要單元測試？</h2>\n<p>我先列出幾個優缺點，我們來比較一下</p>\n<h3>優點：</h3>\n<ol>\n<li>確保團隊跌代的時候不會影響原本的功能</li>\n<li>確保品質

# Supabase: Update Table
- Load Input from NLP Info and Update to Supabase
| nlp result info `field name` | supabase table name `colunm name` | 
| ---------------------------- | ----------------------------------|
| `programming_languages`      | `programming_languages`           |
| `extracted_keywords`         | `keywords_unigram`                |
| `extracted_keywords_phrases` | `keywords_bigram`                 |

In [3]:
# create new client
url = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_API_KEY")
supabase = create_client(url, key)

In [4]:
# test select
'''
test_info_href = test_nlp_info['href']
data = supabase.table("articles").select("*").eq('href', test_info_href).execute()
print(
    json.dumps(
        data.data[0], 
        indent=4, 
        ensure_ascii=False
    )
)
'''
pass


# test update

'''
test_info_href = test_nlp_info['href']

data = supabase.table("articles").update(
    {
        "programming_languages": test_nlp_info['programming_languages'],
        "keywords_unigram":      test_nlp_info['extracted_keywords'],
        "keywords_bigram":       test_nlp_info['extracted_keywords_phrases']
    }
).eq("href", test_info_href).execute()


print(
    json.dumps(
        data.data,
        indent=4, 
        ensure_ascii=False
    )
)
'''
pass

In [5]:
# update all nlp info
for i, nlp_info_href in enumerate(nlp_result_info): 

    nlp_info = nlp_result_info[nlp_info_href]
    
    print("Processing {}/{}".format(i+1, len(nlp_result_info)), end='\r')
    
    data = supabase.table("articles").update(
        {
            "programming_languages": nlp_info['programming_languages'],
            "keywords_unigram":      nlp_info['extracted_keywords'],
            "keywords_bigram":       nlp_info['extracted_keywords_phrases']
        }
    ).eq("href", nlp_info_href).execute()

# Supabase: RPC

- RPC Function for Counting Programming Language Frequency

  ```sql
  CREATE OR REPLACE FUNCTION prog_lang_freq(top_n int)
    
    RETURNS TABLE (prog_lang text, prog_freq int) 
    AS
    $$
      
      WITH prog_lang_flat AS (SELECT UNNEST(programming_languages) AS prog_lang FROM articles)

      SELECT prog_lang, COUNT(*) AS prog_freq
      FROM prog_lang_flat 
      GROUP BY prog_lang
      ORDER BY COUNT(*) DESC, prog_lang
      LIMIT top_n;

    $$ 
    language sql;
  ```

In [6]:
# run the rpc function and show top 10 freq programming language
res_data = supabase.rpc('prog_lang_freq', {'top_n': 10}).execute()
print(json.dumps(res_data.data, indent=4))

[
    {
        "prog_lang": "javascript",
        "prog_freq": 12
    },
    {
        "prog_lang": "golang",
        "prog_freq": 7
    },
    {
        "prog_lang": "html",
        "prog_freq": 7
    },
    {
        "prog_lang": "c",
        "prog_freq": 5
    },
    {
        "prog_lang": "python",
        "prog_freq": 4
    },
    {
        "prog_lang": "sql",
        "prog_freq": 4
    },
    {
        "prog_lang": "c++",
        "prog_freq": 3
    },
    {
        "prog_lang": "css",
        "prog_freq": 3
    },
    {
        "prog_lang": "bash",
        "prog_freq": 2
    },
    {
        "prog_lang": "jsx",
        "prog_freq": 2
    }
]
